In [1]:
"""This function takes in a local json filename and reads it into a single list. It also removes punctuation and null values.
It has default punctuation it keeps "(, ), -, ", but this can be added to.""" 

#TO-DO:
#remove string if only numbers
#document this better, add doc_strings to methods and overall just make better 


import Ruk_Reader
import json
from nltk.tokenize import RegexpTokenizer
import string
import re
from nltk.corpus import stopwords
import glob

def json_cleaner(local_book_title, extra_strings_to_keep = "", print_state = True ):
    
    #this method removes the noise from array
    def _remove_noise(input_text, noise_list):
        if isinstance(input_text, list):
            for word in input_text:
                letters = list(word)
                for letter in letters:
                    if letter in noise_list:
                        letters.remove(letter)
                word = "".join(letters)
                noise_free_words.append(word) 
            return noise_free_words.split()
    
        elif isinstance(input_text, str):
            letters = list(input_text)
            for letter in letters:
                if letter in noise_list:
                    letters.remove(letter)
            noise_free_word = "".join(letters)
            return noise_free_word
    

#Deciding the noise/punctuation to remove from input list
    initial_noise = ["''"] 
    updated_punct = ''.join(i for i in string.punctuation if not i in ['(', ')', '-']) #+ extra_strings_to_keep
    for i in range (0, len(updated_punct)):
        initial_noise.append(updated_punct[i])
    if extra_strings_to_keep:
        initial_noise.remove(extra_strings_to_keep)
        
    
#instantiating lists
    all_words = []
    stripped = []
    stop_words = set(stopwords.words('english'))


#reading file
    with open(local_book_title, "r") as read_file:
        data = json.load(read_file)
    
    print("data is ", type(data))

#looks at keys of dictionary that is the data read. The main content is in the "chap-content"
    print(type(data))
    if isinstance(data, dict):
        subchap_headings = data.keys()
        print("dictionary keys: ", subchap_headings)  #these are probably removable
        print()
        all_content = data.get("chap-content") 
    elif isinstance(data, str):
        all_content = data.split()
    elif isinstance(data, list):
        all_content = data

#Turns list of lists into single list, also removes stopwords, and makes lowercase

    for i in range(0, len(all_content)):
        if all_content[i]:
            all_content[i] = _remove_noise(all_content[i], initial_noise)
            all_content[i] = all_content[i].split()
            for j in range(0, len(all_content[i])):
                if all_content[i][j] and "http" not in all_content[i][j] and all_content[i][j] not in stop_words and all_content[i][j].isdigit() == False:
                    all_words.append(all_content[i][j].lower())
    
    if print_state:
        print(all_words)
    
    else:
        return all_words
    

In [2]:
#json_cleaner(r"C:\Users\bowri\square1\ChemLP\Bowman\textbook_files\Brewing_Science_A_Multidisciplinary_Approach_by_Mosher_and_Trantham.json")

In [3]:
"""This function takes in a bunch of things and creates a word2vec CBOW NN"""

import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency
import spacy  # For preprocessing
from spacy.lang.en import English  #not sure what this is for, check if needed
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)
import multiprocessing
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser  #for bigrams/common phrase ID



def word2vec_book_model(list_of_book_words, min_count=2, window = 2, size = 300, sample = 6e-5, alpha=0.03
                       ,min_alpha = 0.0007, negative = 0, workers = 6):

    phrases = Phrases(list_of_book_words, min_count=30, progress_per=10000)   
    bigram = Phraser(phrases)
    sentences = bigram[list_of_book_words]
    
    cores = multiprocessing.cpu_count() 

#1) INITALIZING THE MODEL
    w2v_model = Word2Vec(min_count = min_count,         #Ignores all words with total absolute frequency lower than this - (2, 100)
                     window = window,           #The maximum distance between the current and predicted word within a sentence.
                     size = size,           #size of feature vectors
                     sample = sample,   #threshold for configuring which higher-frequency words are randomly downsampled. influencial. (0, 1e-5)
                     alpha = alpha,        #learning rate initial (0.01 - 0.05)
                     min_alpha = min_alpha,     #final learning rate, linearly drops to this as training goes
                     negative = negative,  # If > 0,  how many "noise words" should be drown. If set to 0, no negative sampling is used. - (5, 20)
                     workers = workers)        #how many cores doing the work

#2) BUILDING VOCAB
    t = time()
    w2v_model.build_vocab(sentences, progress_per=10000)
    print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))
    
#3) TRAINING MODEL

    #    total_examples = int, Count of sentences;
    #   epochs = int - Number of iterations (epochs) over the corpus - [10, 20, 30]
    
    t = time()
    print("length of total examples is ", len(sentences))
    w2v_model.train(sentences, total_examples = len(sentences), epochs=30, report_delay=1)
    print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))


    w2v_model.init_sims(replace=True)  #do this when done training the model? normalizes the length of the vectors before comparing
    
    print("the size of the vocabulary is: ", len(w2v_model.wv.vocab))
    print("size of the corpus_count:", w2v_model.corpus_count)  #
    return w2v_model

INFO - 10:13:55: 'pattern' package not found; tag filters are not available for English


In [4]:
#turn this to a function that can be used to create a super list for full pckg for word2vec .py file

def bow_list_maker():   #add default string to take in for additional characters to keep
    """This function is similar to super list maker, and the feed2vec function, but instead of creating a dataframe, the function creates a 
    list of lists, where each list is all the cleaned words from a book. It calls json_cleaner defined above"""
    
    initPath = input("Enter a file location")
    bookType = input("Enter a data type(.json please)")
    token_list = []
    final_list = []
    if bookType:
        path = initPath + "\*" + bookType
    else:
        path = initPath
    raw_path = r"{}".format(path)
    bookList = glob.glob(raw_path)
    
    yesOrNo = input("Is there an additional file type? (y or n) ")
    if yesOrNo == "y":
        bookType2 = input("Enter a second data type(.txt) ")
        path2 = initPath + "\*" + bookType2
        raw_path2 = r"{}".format(path2)
        bookList2 = glob.glob(raw_path2)
        for bookSite in bookList2:
            bookList.append(bookSite)
            
    print("book list is ", bookList)
    Multiple_book_list = []
    #local_titles = ["Principles_of_Polymer_Chemistry_by_Ravve.json", "Schaller_polym_chem", "Soderberg_bio_o_chem"]
    
    for title in bookList:
        list_of_words = json_cleaner(title, "", False)
        Multiple_book_list.append(list_of_words)
        print(len(Multiple_book_list))
        print("length of ", title, "is ", len(list_of_words))
    
    for book in Multiple_book_list:
        print(len(book))
        print(book[:100])
        
    return Multiple_book_list

    #"C:\Users\bowri\square1\ChemLP\Bowman\textbook_files\Brewing_Science_A_Multidisciplinary_Approach_by_Mosher_and_Trantham.json"

In [5]:
#word2vec_book_model(Multiple_book_list, min_count = 5, window = 3)
#print(len(Multiple_book_list))
list_of_local_books = bow_list_maker()
three_book_model = word2vec_book_model(list_of_local_books, min_count = 10, window = 4, size = 400, sample = 6e-5, negative = 10)


Enter a file location C:\Users\bowri\square1\ChemLP\Bowman\textbook_files
Enter a data type(.json please) .json
Is there an additional file type? (y or n)  y
Enter a second data type(.txt)  .txt


book list is  ['C:\\Users\\bowri\\square1\\ChemLP\\Bowman\\textbook_files\\Brewing_Science_A_Multidisciplinary_Approach_by_Mosher_and_Trantham.json', 'C:\\Users\\bowri\\square1\\ChemLP\\Bowman\\textbook_files\\Polymer_Chemistry_by_Koltzsenburg_Maskos_and_Nuyken.json', 'C:\\Users\\bowri\\square1\\ChemLP\\Bowman\\textbook_files\\Polymer_Synthesis_Theory_and_Practice_by_Braun_Cherdron_Rehahn_Ritter_and_Voit.json', 'C:\\Users\\bowri\\square1\\ChemLP\\Bowman\\textbook_files\\Book Basic Principles of Organic Chemistry (Roberts and Caserio).txt', 'C:\\Users\\bowri\\square1\\ChemLP\\Bowman\\textbook_files\\Book Logic of Organic Synthesis (Rao).txt', 'C:\\Users\\bowri\\square1\\ChemLP\\Bowman\\textbook_files\\Book Organic Chemistry - A Carbonyl Early Approach (McMichael).txt', 'C:\\Users\\bowri\\square1\\ChemLP\\Bowman\\textbook_files\\Soderberg_bio_o_chem.txt']
data is  <class 'list'>
<class 'list'>
1
length of  C:\Users\bowri\square1\ChemLP\Bowman\textbook_files\Brewing_Science_A_Multidiscipl

INFO - 10:14:09: collecting all words and their counts
INFO - 10:14:09: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 10:14:10: collected 441496 word types from a corpus of 614796 words (unigram + bigrams) and 7 sentences
INFO - 10:14:10: using 441496 counts as vocab in Phrases<0 vocab, min_count=30, threshold=10.0, max_vocab_size=40000000>
INFO - 10:14:10: source_vocab length 441496
INFO - 10:14:14: Phraser built with 452 phrasegrams
INFO - 10:14:14: collecting all words and their counts
INFO - 10:14:14: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 10:14:15: collected 46980 word types from a corpus of 581252 raw words and 7 sentences
INFO - 10:14:15: Loading a fresh vocabulary
INFO - 10:14:15: effective_min_count=10 retains 6851 unique words (14% of original 46980, drops 40129)
INFO - 10:14:15: effective_min_count=10 leaves 501142 word corpus (86% of original 581252, drops 80110)
INFO - 10:14:15: deleting the raw counts dictionary of 4

Time to build vocab: 0.02 mins
length of total examples is  7


INFO - 10:14:15: training model with 6 workers on 6851 vocabulary and 400 features, using sg=0 hs=0 sample=6e-05 negative=10 window=4
INFO - 10:14:16: worker thread finished; awaiting finish of 5 more threads
INFO - 10:14:16: worker thread finished; awaiting finish of 4 more threads
INFO - 10:14:16: worker thread finished; awaiting finish of 3 more threads
INFO - 10:14:16: worker thread finished; awaiting finish of 2 more threads
INFO - 10:14:16: EPOCH 1 - PROGRESS: at 85.71% examples, 49403 words/s, in_qsize 1, out_qsize 1
INFO - 10:14:16: worker thread finished; awaiting finish of 1 more threads
INFO - 10:14:16: worker thread finished; awaiting finish of 0 more threads
INFO - 10:14:16: EPOCH - 1 : training on 581252 raw words (66610 effective words) took 1.2s, 57384 effective words/s
INFO - 10:14:17: worker thread finished; awaiting finish of 5 more threads
INFO - 10:14:17: worker thread finished; awaiting finish of 4 more threads
INFO - 10:14:17: worker thread finished; awaiting fin

Time to train the model: 0.63 mins


INFO - 10:14:53: precomputing L2-norms of word weight vectors


the size of the vocabulary is:  6851
size of the corpus_count: 7


In [6]:
#write a quick method here for simplifying the below

def model_analyzer(word2vec_model, word1_compare, word2_compare, similarity_word = ""):
    #print("similarity between alchohol and hydroxyl is: ", new_trained_model.wv.similarity("alcohol", "hydroxyl"))
    compare_num =  (word2vec_model.wv.similarity(word1_compare, word2_compare))
    print("similarity between", word1_compare, " and ", word2_compare, " is  ", word2vec_model.wv.similarity(word1_compare, word2_compare))

     
    if similarity_word != "":
        print("words most similar to", similarity_word, "are: ")
        print(word2vec_model.wv.most_similar(positive=[similarity_word], topn = 10))

    return compare_num


model_analyzer(three_book_model, "alcohol", "hydroxyl", similarity_word = "carbon")

similarity between alcohol  and  hydroxyl  is   0.7402404
words most similar to carbon are: 
[('attached', 0.9968094825744629), ('carbons', 0.9901885986328125), ('hydrogens', 0.9870703816413879), ('carbon_atom', 0.9869042038917542), ('oxygen', 0.9862474799156189), ('hydrogen_atoms', 0.985282838344574), ('bonded', 0.9836483001708984), ('double_bond', 0.9827227592468262), ('breaking', 0.9747698903083801), ('four', 0.9737505912780762)]


0.7402404

In [7]:
def syn_ant_tester(nn_model):

    elec_nuc = model_analyzer(nn_model, "electrophile", "nucleophile")   #opposites
    acid_base = model_analyzer(nn_model,"acid", "base")      #opposites
    polar_nonpol = model_analyzer(nn_model,"polar", "nonpolar")   #opposites
    synth_nat = model_analyzer(nn_model,"synthetic", "natural")   #opposites
    pos_neg = model_analyzer(nn_model,"positive", "negative")   #opposites
    #fs = model_analyzer(nn_model,"fast", "slow")   #opposites
    #db = model_analyzer(nn_model,"dark", "bright")   #opposites
    sc = model_analyzer(nn_model,"simple", "complex")   #opposites
    #mc = model_analyzer(nn_model,"polyamides", "ligand")   #opposites

    ant_avg = (elec_nuc+acid_base+polar_nonpol+synth_nat+pos_neg+sc)/6
    print("ant avg: ", ant_avg)
    print()
    #print(type(nn_model.wv.vocab))
    cn = model_analyzer(nn_model,"carbon", "nitrogen")  #related- same ish
    kc = model_analyzer(nn_model,"ketone", "carbonyl") #same ish 
    ah = model_analyzer(nn_model,"alcohol", "hydroxyl")  #same
    ph = model_analyzer(nn_model,"proton", "hydrogen")  #same  
    pc = model_analyzer(nn_model,"polymer", "chain")  #same
    en = model_analyzer(nn_model,"electron", "negative")  #same
    #hw = model_analyzer(nn_model,"water", "h2o")  #same
    

    syn_avg = (cn+kc+ah+ph+pc+en)/6
    print("syn avg: ", syn_avg)


syn_ant_tester(three_book_model)



similarity between electrophile  and  nucleophile  is   0.8939026
similarity between acid  and  base  is   0.93316615
similarity between polar  and  nonpolar  is   0.94328654
similarity between synthetic  and  natural  is   0.96645224
similarity between positive  and  negative  is   0.99771947
similarity between simple  and  complex  is   0.9166586
ant avg:  0.9418641726175944

similarity between carbon  and  nitrogen  is   0.9730783
similarity between ketone  and  carbonyl  is   0.90407205
similarity between alcohol  and  hydroxyl  is   0.7402404
similarity between proton  and  hydrogen  is   0.8267946
similarity between polymer  and  chain  is   0.6491602
similarity between electron  and  negative  is   0.8734058
syn avg:  0.8277919292449951


In [8]:

#word2vec_book_model(min_count = 3, window = 4, size = 300, sample = 6e-5, negative = 10) - good settings from capstone



In [9]:
syn_ant_tester(Ravve_Soder)
#print(Schaller_Model.wv.vocab)

NameError: name 'Ravve_Soder' is not defined